In [ ]:
import datetime
import numpy
import pygetm

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain with outer points masked
domain = pygetm.Domain.create_cartesian(500.*numpy.arange(100), 500.*numpy.arange(30), 1, periodic_x=True)
domain.T.H[1:-1, :] = 50.
domain.T.mask[1:-1, :] = 1
sim = pygetm.Simulation(domain, runtype=1, advection_scheme=1)
domain.U.H[1:-1, :] = 50.
domain.U.mask[1:-1, :] = 1

# Idealized surface forcing
tausx, tausx_ = domain.T.array(fill=0.)
tausx[...] = 0.01
tausy_ = numpy.zeros_like(tausx_)
sp, sp_ = domain.T.array(fill=0.)

# Time
start = datetime.datetime(2020, 1, 1, 0, 0)
stop = datetime.datetime(2020, 1, 1, 2, 0)
timestep = 10.
time = start

# Set up velocity plot
nsample = 4
fig = matplotlib.pyplot.figure()
ax = fig.gca()
pc = ax.pcolormesh(domain.T.xi, domain.T.yi, domain.T.z, vmin=-0.001, vmax=0.001)
cb = fig.colorbar(pc)
cb.set_label('elevation (m)')
Q = ax.quiver(domain.T.x[::nsample, ::nsample], domain.T.y[::nsample, ::nsample], sim.momentum.U[::nsample, ::nsample], sim.momentum.V[::nsample, ::nsample], scale=.3)
title = ax.set_title(time.strftime('%H:%M:%S'))

In [ ]:
dist_U = domain.distribute(sim.momentum.U_)
dist_V = domain.distribute(sim.momentum.V_)
dist_U.update_halos()
dist_V.update_halos()
plotting_interval = 5
istep = 0
while time < stop:
    istep += 1
    print(istep, sim.momentum.U[1:-1,:].min(), sim.momentum.U[1:-1,:].max())
    
    sim.pressure.surface(domain.T.z_, sp_)
    sim.momentum.uv_momentum_2d(timestep, tausx_, tausy_, sim.pressure.dpdx_, sim.pressure.dpdy_)
    dist_U.update_halos()
    dist_V.update_halos()
    sim.sealevel.update(timestep, sim.momentum.U_, sim.momentum.V_)
    domain.depth_update()

    # Update velocity plot
    if istep % plotting_interval == 0:
        Q.set_UVC(sim.momentum.U[::nsample, ::nsample], sim.momentum.V[::nsample, ::nsample])
        title.set_text(time.strftime('%H:%M:%S'))
        pc.set_array(domain.T.z.ravel())
        fig.canvas.draw()

    time += datetime.timedelta(seconds=timestep)
    